In [7]:
# setting environment variables
import os
os.environ["JAVA_HOME"] = "/opt/anaconda3/envs/bi_development"  #--> '/opt/anaconda3/envs/bi_development/bin/java'
os.environ["SPARK_HOME"] = "/home/spark/spark-3.1.2-bin-hadoop2.7"

In [9]:
import findspark
findspark.init()

In [13]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import datetime


conf = pyspark.SparkConf().setAppName('JupyterHub').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)


dt_ini = datetime.datetime.now()

df_sp = spark.read.parquet('/mnt/lnxsasdtcn/sasdata/DISCO_M/ALZFenix/local/FtpPCSas/history_azfd_nov2019_sas')

dt_end = datetime.datetime.now()
print(dt_end - dt_ini)

df_sp.head(1)

0:00:06.833709


[Row(OE='AZFD', CLAIM_CODE='340004286', FECHA_PROCESO=datetime.datetime(2006, 12, 1, 0, 0), ACCIDENT_DATE=datetime.datetime(1992, 4, 1, 0, 0), REPORTING_DATE=datetime.datetime(1992, 4, 1, 0, 0), CURRENCY='EUR', GUARANTEE='30', PORTFOLIO_LOB='1020', RESERVING_LOB='1020_BI', G_PAID=0.0, G_SS=0.0, G_CASE=6000.0, G_CASE_RECOVERY=0.0, C_PAID=0.0, C_SS=0.0, C_CASE=0.0, FLAG='1', FIRST_CLOSE_DATE='', LATEST_CLOSE_DATE='', NATCAT='0', INSTRUMENTAL='0', CONSORCIABLE='0')]

In [50]:
convert_to_month = udf(lambda x:x.strftime('%Y%m') )
%time df_month = df_sp.withColumn("PROCESS_MONTH", convert_to_month(df_sp["FECHA_PROCESO"]))

#At the end, this timezones conversion aren't needed. Pyspark takes the right timezone from the parquet file by default.
df_2 = df_month.withColumn("PROCESS_MONTH", convert_to_month(from_utc_timestamp(split(df_month["FECHA_PROCESO"],'\+')[0],'GMT-1')))
df_2 = df_2.withColumn("FECHA_PROCESO", from_utc_timestamp(split(df_2["FECHA_PROCESO"],'\+')[0],'GMT-1')) 
df_2 = df_2.withColumn("ACCIDENT_DATE", from_utc_timestamp(split(df_2["ACCIDENT_DATE"],'\+')[0],'GMT-2')) 
df_2 = df_2.withColumn("REPORTING_DATE", from_utc_timestamp(split(df_2["REPORTING_DATE"],'\+')[0],'GMT-2')) 


df_month.head(1)

CPU times: user 536 µs, sys: 3.38 ms, total: 3.92 ms
Wall time: 14.8 ms


[Row(OE='AZFD', CLAIM_CODE='340004286', FECHA_PROCESO=datetime.datetime(2006, 12, 1, 0, 0), ACCIDENT_DATE=datetime.datetime(1992, 4, 1, 0, 0), REPORTING_DATE=datetime.datetime(1992, 4, 1, 0, 0), CURRENCY='EUR', GUARANTEE='30', PORTFOLIO_LOB='1020', RESERVING_LOB='1020_BI', G_PAID=0.0, G_SS=0.0, G_CASE=6000.0, G_CASE_RECOVERY=0.0, C_PAID=0.0, C_SS=0.0, C_CASE=0.0, FLAG='1', FIRST_CLOSE_DATE='', LATEST_CLOSE_DATE='', NATCAT='0', INSTRUMENTAL='0', CONSORCIABLE='0', PROCESS_MONTH='200612')]

In [ ]:
%time df_month.describe()

In [53]:
%time df_month.dtypes

CPU times: user 46 µs, sys: 0 ns, total: 46 µs
Wall time: 49.1 µs


[('OE', 'string'),
 ('CLAIM_CODE', 'string'),
 ('FECHA_PROCESO', 'timestamp'),
 ('ACCIDENT_DATE', 'timestamp'),
 ('REPORTING_DATE', 'timestamp'),
 ('CURRENCY', 'string'),
 ('GUARANTEE', 'string'),
 ('PORTFOLIO_LOB', 'string'),
 ('RESERVING_LOB', 'string'),
 ('G_PAID', 'double'),
 ('G_SS', 'double'),
 ('G_CASE', 'double'),
 ('G_CASE_RECOVERY', 'double'),
 ('C_PAID', 'double'),
 ('C_SS', 'double'),
 ('C_CASE', 'double'),
 ('FLAG', 'string'),
 ('FIRST_CLOSE_DATE', 'string'),
 ('LATEST_CLOSE_DATE', 'string'),
 ('NATCAT', 'string'),
 ('INSTRUMENTAL', 'string'),
 ('CONSORCIABLE', 'string'),
 ('PROCESS_MONTH', 'string')]

In [54]:
%time df_month.schema

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.82 µs


StructType(List(StructField(OE,StringType,true),StructField(CLAIM_CODE,StringType,true),StructField(FECHA_PROCESO,TimestampType,true),StructField(ACCIDENT_DATE,TimestampType,true),StructField(REPORTING_DATE,TimestampType,true),StructField(CURRENCY,StringType,true),StructField(GUARANTEE,StringType,true),StructField(PORTFOLIO_LOB,StringType,true),StructField(RESERVING_LOB,StringType,true),StructField(G_PAID,DoubleType,true),StructField(G_SS,DoubleType,true),StructField(G_CASE,DoubleType,true),StructField(G_CASE_RECOVERY,DoubleType,true),StructField(C_PAID,DoubleType,true),StructField(C_SS,DoubleType,true),StructField(C_CASE,DoubleType,true),StructField(FLAG,StringType,true),StructField(FIRST_CLOSE_DATE,StringType,true),StructField(LATEST_CLOSE_DATE,StringType,true),StructField(NATCAT,StringType,true),StructField(INSTRUMENTAL,StringType,true),StructField(CONSORCIABLE,StringType,true),StructField(PROCESS_MONTH,StringType,true)))

In [20]:
%time df_month.groupBy("PROCESS_MONTH").count().show()

+-------------+------+
|PROCESS_MONTH| count|
+-------------+------+
|       200206| 32035|
|       201103| 68401|
|       201809|156970|
|       200606| 43669|
|       201503| 80721|
|       201607|129829|
|       200203| 23851|
|       201903| 90771|
|       200106| 43229|
|       201702| 77188|
|       201510|149513|
|       201909|150890|
|       201611|173475|
|       200209| 39162|
|       201801| 66089|
|       201808|146777|
|       201009|117035|
|       200706| 66218|
|       201306| 98067|
|       200809| 97504|
+-------------+------+
only showing top 20 rows

CPU times: user 18.6 ms, sys: 10.1 ms, total: 28.7 ms
Wall time: 3min 33s


In [56]:
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")

In [68]:
groups = [x[0] for x in df_month.select("PROCESS_MONTH").distinct().collect()]

In [70]:
len(groups)

127

In [71]:
dt_ini = datetime.datetime.now()   

groups = [x[0] for x in df_month.select("PROCESS_MONTH").distinct().collect()]
for month in groups:
    df_month.filter(df_month["PROCESS_MONTH"]==month).\
    repartition(1).write.csv(r'data/fenix_file_{}'.format(month),\
                             compression="gzip", mode="overwrite", header=True,sep = ';')

dt_end = datetime.datetime.now() 

print(dt_end - dt_ini)

4:46:41.561663


In [74]:
spark.stop()